In [1]:
from seldon_deploy_sdk import KubernetesResourcesApi, Configuration, ApiClient, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition, PredictApi
from seldon_deploy_sdk.auth import OIDCAuthenticator
from seldon_deploy_sdk.rest import ApiException

import pprint

In [4]:
SD_IP = "35.196.51.203"
config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = 'client_credentials'
config.verify_ssl = False
auth = OIDCAuthenticator(config)
config.id_token = auth.authenticate()
seldon_api_client = ApiClient(configuration=config, authenticator=auth)

predict_api = PredictApi(seldon_api_client)
deployment_api = SeldonDeploymentsApi(seldon_api_client)
kubernetes_api = KubernetesResourcesApi(seldon_api_client)

You can now send requests to your model. As an example of a normal request:
```
{
    "data": {
        "names": ["step", "type", "amount", "oldBalanceOrig", "newBalanceOrig",
                  "oldBalanceDest", "newBalanceDest", "errorBalanceOrig", "errorBalanceDest"],
        "ndarray": [
            [205, 1, 63243.44, -1.00, -1.00, 1853683.32, 1916926.76, 63243.44, 0]
        ]
    }
}
```
And a fraudulent transaction too:

```
{
    "data": {
        "names": ["step", "type", "amount", "oldBalanceOrig", "newBalanceOrig",
                  "oldBalanceDest", "newBalanceDest", "errorBalanceOrig", "errorBalanceDest"],
        "ndarray": [
            [629, 1, 2433009.28, 2433009.28, 0.00, 0.00, 2433009.28, 0.00, 0.00]
        ]
    }
}
```

In [8]:
MODEL_NAME = "iris-classification"
DEPLOYMENT_NAME = f"{MODEL_NAME}"
NAMESPACE = "seldon-demos"

In [13]:
# create an instance of the API class
prediction = {
    "data": {
        "names": ["Sepal length","Sepal width","Petal length", "Petal Width"],
        "ndarray": [
            [6.8,  2.8,  4.8,  1.4],
            [6.1,  3.4,  4.5,  1.6]
        ]
    }
}

try:
    api_response = predict_api.predict_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE, prediction)
    pprint.pprint(api_response)
except ApiException as e:
    print("Exception when calling PredictApi->predict_seldon_deployment: %s\n" % e)

{'data': {'names': ['t:0', 't:1', 't:2'],
          'ndarray': [[0.008074020139119254,
                       0.7781601484223358,
                       0.2137658314385449],
                      [0.04569799579421536,
                       0.5165292130301902,
                       0.4377727911755945]]},
 'meta': {'requestPath': {'iris-classification-container': 'seldonio/sklearnserver:1.13.1'}}}


In [ ]:
try:
    deployment_status = deployment_api.validate_seldon_deployment(NAMESPACE, DEPLOYMENT_NAME)
    print(api_response)
except ApiException as e:
    print("Exception when calling SeldonDeploymentsApi->validate_seldon_deployment: %s\n" % e)

In [ ]:
try:
    api_response = deployment_api.read_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE)
    print(api_response)
except ApiException as e:
    print("Exception when calling SeldonDeploymentsApi->read_seldon_deployment: %s\n" % e)


In [ ]:
name = 'blabla' # str | Name identifies a resource
namespace = 'seldon-demos' # str | Namespace provides a logical grouping of resources
component = 'predictor' # str | Component differentiates between types of model (e.g. predictor, explainer... etc) (optional)

try:
    api_response = kubernetes_api.list_seldon_deployment_resources(DEPLOYMENT_NAME, NAMESPACE, component=component)
    print(api_response)
except ApiException as e:
    print("Exception when calling KubernetesResourcesApi->list_seldon_deployment_resources: %s\n" % e)